# Scraping Birds
Performed actions:
1. Create driver
2. Go to base website
3. For each bird:
    * Go to dedicated website
    * Scrape information
    * Go back to base website
4. Quit driver
5. Export csv

In [1]:
### setup

from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

import pandas as pd
import time
import random

birds = pd.read_csv('birds.csv')["name"].tolist()

latin_names = list()
foods = list()
habitats = list()
nest_sites = list()

In [2]:
### scrape

# create driver
service = Service("/Users/Alain/chromedriver_mac64/chromedriver")
driver = webdriver.Chrome(service=service)
driver.implicitly_wait(10)

# go to base website
base_URL = "https://www.vogelwarte.ch/en/birds/birds-of-switzerland/"
driver.get(base_URL)

for bird in birds:

      # download content
      search = driver.find_elements(By.TAG_NAME, "input")[11]
      search.send_keys(bird)
      time.sleep(random.uniform(0.1, 0.2))

      search.send_keys(Keys.ENTER)
      time.sleep(random.uniform(0.1, 0.2))

      try:
            # find features
            web_page = driver.page_source
            soup = BeautifulSoup(web_page, "lxml")

            latin_name = soup.find_all("span")[13].text
            characteristics = soup.find_all("div", class_="vds-col")
            food = characteristics[9].text
            habitat = characteristics[11].text
            nest_site = characteristics[15].text

            driver.back()
            time.sleep(random.uniform(0.1, 0.2))
      except:
            latin_name = ""
            characteristics = ""
            food = ""
            habitat = ""
            nest_site = ""

            driver.get(base_URL)
            time.sleep(random.uniform(0.1, 0.2))

      # store features
      latin_names.append(latin_name)
      foods.append(food)
      habitats.append(habitat)
      nest_sites.append(nest_site)

driver.quit()

IndexError: list index out of range

In [ ]:
### raw scraped dataset

df = pd.DataFrame({'English Name' : birds,'Latin Name': latin_names, 'Food': foods, 'Habitat': habitats, 'Nest Site': nest_sites})
df.to_csv('birds_data.csv', index=False)